In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
csv_file = "listings-3.csv"
airbnb_df = pd.read_csv(csv_file)
airbnb_df.head()

/Users/afreites/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1419,https://www.airbnb.com/rooms/1419,20190307044035,43531,Beautiful home in amazing area!,"This large, family home is located in one of T...",Visit Toronto with this fabulous furnished hom...,"This large, family home is located in one of T...",none,The apartment is located in the Ossington stri...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.16
1,8077,https://www.airbnb.com/rooms/8077,20190307044035,43531,Downtown Harbourfront Private Room,Guest room in a luxury condo with access to al...,My husband and I are both artists. I give pain...,Guest room in a luxury condo with access to al...,none,NaN,...,t,f,strict_14_with_grace_period,f,t,2,1,1,0,1.45
2,12604,https://www.airbnb.com/rooms/12604,20190307044035,43531,Seaton Village Parlour Bedroom,Comfortable sofa bed in airy main floor parlour.,Family of 4 (2 kids) plus a friendly dog open...,Comfortable sofa bed in airy main floor parlou...,none,NaN,...,f,f,moderate,f,f,1,0,1,0,NaN
3,23691,https://www.airbnb.com/rooms/23691,20190307044035,43531,Queen Bedroom close to downtown,NaN,Nestled in the middle of a quiet residential n...,Nestled in the middle of a quiet residential n...,none,NaN,...,t,f,strict_14_with_grace_period,f,f,2,0,2,0,1.94
4,26654,https://www.airbnb.com/rooms/26654,20190307044035,43531,"World Class downtown@ CN Tower! Jays, TIFF, ACC","CN Tower, TIFF Bell Lightbox, Metro Convention...",Welcome to 5***** Living - Bohemian decor~ in ...,"CN Tower, TIFF Bell Lightbox, Metro Convention...",none,NaN,...,f,f,strict_14_with_grace_period,f,f,7,7,0,0,0.36


In [4]:
airbnb_data_df = airbnb_df[['zipcode','property_type', 'room_type','price']].copy()


In [5]:
airbnb_data_df = airbnb_data_df.rename(columns={'zipcode': 'postal_code'})
airbnb_data_df.head()

,postal_code,property_type,room_type,price
0,M6J,House,Entire home/apt,469
1,M5J,Apartment,Private room,100
2,M6G,House,Private room,67
3,M6E 2P3,House,Private room,70
4,M5V 3P5,Condominium,Entire home/apt,200


In [6]:
airbnb_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20036 entries, 0 to 20035
Data columns (total 4 columns):
postal_code      19651 non-null object
property_type    20036 non-null object
room_type        20036 non-null object
price            20036 non-null int64
dtypes: int64(1), object(3)
memory usage: 626.2+ KB


In [7]:
airbnb_data_df.describe()

,price
count,20036.000000
mean,138.044620
std,208.187617
min,0.000000
25%,66.000000
50%,100.000000
75%,157.000000
max,13395.000000


In [8]:
airbnb_data_df.dtypes

postal_code      object
property_type    object
room_type        object
price             int64
dtype: object

In [9]:
airbnb_data_df.postal_code.unique()

array(['M6J', 'M5J', 'M6G', ..., 'M1W 2J3', 'M2L 1Z5', 'M4V 2B7'],
      dtype=object)

In [10]:
airbnb_clean_df = airbnb_data_df.loc[airbnb_data_df["postal_code"] .str.len() == 7, [
    "property_type", "room_type", "price","postal_code"]]
airbnb_clean_df.head()

,property_type,room_type,price,postal_code
3,House,Private room,70,M6E 2P3
4,Condominium,Entire home/apt,200,M5V 3P5
5,Apartment,Entire home/apt,54,M4L 2P8
6,Apartment,Entire home/apt,80,M4S 2M5
7,Apartment,Entire home/apt,134,M5J 2V3


In [11]:
airbnb_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18010 entries, 3 to 20035
Data columns (total 4 columns):
property_type    18010 non-null object
room_type        18010 non-null object
price            18010 non-null int64
postal_code      18010 non-null object
dtypes: int64(1), object(3)
memory usage: 703.5+ KB


In [18]:
airbnb_transformed= airbnb_clean_df[['postal_code','property_type', 'room_type','price']].copy()
airbnb_transformed.drop_duplicates("postal_code", inplace=True)
airbnb_transformed.set_index("postal_code", inplace=True)

airbnb_transformed.head()

,property_type,room_type,price
postal_code,,,
M6E 2P3,House,Private room,70
M5V 3P5,Condominium,Entire home/apt,200
M4L 2P8,Apartment,Entire home/apt,54
M4S 2M5,Apartment,Entire home/apt,80
M5J 2V3,Apartment,Entire home/apt,134
